In [4]:
ls

 Volume in drive C has no label.
 Volume Serial Number is B483-79FB

 Directory of C:\Users\arnol\OneDrive\Documents\SMU\GitSMU\python-api-challenge\VacationPy

08/09/2023  05:00 PM    <DIR>          .
08/08/2023  03:37 PM    <DIR>          ..
08/09/2023  04:42 PM    <DIR>          .ipynb_checkpoints
08/09/2023  05:00 PM           119,997 VacationPy.ipynb
               1 File(s)        119,997 bytes
               3 Dir(s)  890,968,825,856 bytes free


# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mount pearl,47.5166,-52.7813,61.32,80,100,11.50,CA,1691603004
1,1,coquimbo,-29.9533,-71.3436,60.53,77,75,8.05,CL,1691602995
2,2,waitangi,-43.9535,-176.5597,47.82,83,34,5.99,NZ,1691603005
3,3,mahina,-17.5065,-149.4890,73.17,83,20,1.14,PF,1691602942
4,4,mao,39.8885,4.2658,81.16,69,0,8.05,ES,1691603005


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:

# Configure the map plot
map = city_data_df.hvplot.points("Lng","Lat",alpha=0.1,color="green",
                                 geo=True, #xlim=(-180, 180), ylim=(-60, 80),
                                 coastline=True)
map

C:\Users\arnol\anaconda3\envs\pyviz\lib\site-packages\cartopy\crs.py:245: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
C:\Users\arnol\anaconda3\envs\pyviz\lib\site-packages\cartopy\crs.py:297: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
C:\Users\arnol\anaconda3\envs\pyviz\lib\site-packages\cartopy\crs.py:364: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:
C:\Users\arnol\anaconda3\envs\pyviz\lib\site-packages\cartopy\cr

:Overlay
   .Points.I    :Points   [Lng,Lat]
   .Coastline.I :Feature   [Longitude,Latitude]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
Best_cities = city_data_df.loc[(city_data_df["Max Temp"] < 90) & (city_data_df["Max Temp"] > 70) &
                 (city_data_df["Lat"] > -25) & (city_data_df["Lat"] < 30) &
                 (city_data_df["Lng"] > -130) & (city_data_df["Lng"] < 150) &
                (city_data_df["Humidity"] > 70) & (city_data_df["Cloudiness"] > 0)] 

Best_cities= Best_cities.dropna()
Best_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,hithadhoo,-0.6000,73.0833,81.45,73,87,3.47,MV,1691603008
24,24,mangrol,21.1167,70.1167,80.56,83,100,15.39,IN,1691603013
34,34,gorontalo,0.5412,123.0595,72.90,98,96,2.98,ID,1691603016
41,41,hadibu,12.6500,54.0333,79.23,75,93,17.27,YE,1691603018
42,42,nova sintra,14.8667,-24.7167,73.42,72,8,15.70,CV,1691603019
...,...,...,...,...,...,...,...,...,...,...
682,682,baturaja,-4.1333,104.1667,72.30,94,94,2.73,ID,1691603247
683,683,zungeru,9.8128,6.1558,84.18,89,83,4.09,NG,1691603247
703,703,ceeldheer,3.8488,47.1806,76.17,83,53,16.84,SO,1691603254
713,713,pipili,20.1167,85.8333,84.54,89,40,2.30,IN,1691603257


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Best_cities[["City", "Country","Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df




,City,Country,Lat,Lng,Humidity,Hotel Name
10,hithadhoo,MV,-0.6000,73.0833,73,
24,mangrol,IN,21.1167,70.1167,83,
34,gorontalo,ID,0.5412,123.0595,98,
41,hadibu,YE,12.6500,54.0333,75,
42,nova sintra,CV,14.8667,-24.7167,72,
...,...,...,...,...,...,...
682,baturaja,ID,-4.1333,104.1667,94,
683,zungeru,NG,9.8128,6.1558,89,
703,ceeldheer,SO,3.8488,47.1806,83,
713,pipili,IN,20.1167,85.8333,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel

params = {
    "categories":"accommodation.hotel",
    "apiKey" :"8dcd74f28da9412a8ae4a412180325d1"
            }


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},10000"
    params["bias"] = f"proximity:{Lng},{Lat}" 
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty and convert to JSON
    name_address = requests.get(base_url, params=params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hithadhoo - nearest hotel: Pebbles Inn
mangrol - nearest hotel: No hotel found
gorontalo - nearest hotel: Grand City Hotel
hadibu - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
axim - nearest hotel: Axim Beach Hotel
hengchun - nearest hotel: 海的顏色精品旅館
west island - nearest hotel: Cocos Village Bungalows
bamboo flat - nearest hotel: Hotel Megapode Nest
georgetown - nearest hotel: Page 63 hostel
tanjung pandan - nearest hotel: Billiton Hotel & Club
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
pasuquin - nearest hotel: No hotel found
inarajan village - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
tabou - nearest hotel: hôtel le rochet
raipur - nearest hotel: Tillu Hotel
myeik - nearest hotel: Green Eyes Hotel
gemena - nearest hotel: La Joie
pimentel - nearest hotel: Hospedaje Calle del Sol
abepura - nearest hotel: Yoyo
natal - nearest hotel: Natal Palace Hotel
gamba - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
10,hithadhoo,MV,-0.6000,73.0833,73,Pebbles Inn
24,mangrol,IN,21.1167,70.1167,83,No hotel found
34,gorontalo,ID,0.5412,123.0595,98,Grand City Hotel
41,hadibu,YE,12.6500,54.0333,75,No hotel found
42,nova sintra,CV,14.8667,-24.7167,72,Residência Ka Dencho
...,...,...,...,...,...,...
682,baturaja,ID,-4.1333,104.1667,94,No hotel found
683,zungeru,NG,9.8128,6.1558,89,No hotel found
703,ceeldheer,SO,3.8488,47.1806,83,No hotel found
713,pipili,IN,20.1167,85.8333,89,No hotel found


In [22]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,hithadhoo,MV,-0.6000,73.0833,73,Pebbles Inn
24,mangrol,IN,21.1167,70.1167,83,No hotel found
34,gorontalo,ID,0.5412,123.0595,98,Grand City Hotel
41,hadibu,YE,12.6500,54.0333,75,No hotel found
42,nova sintra,CV,14.8667,-24.7167,72,Residência Ka Dencho
...,...,...,...,...,...,...
682,baturaja,ID,-4.1333,104.1667,94,No hotel found
683,zungeru,NG,9.8128,6.1558,89,No hotel found
703,ceeldheer,SO,3.8488,47.1806,83,No hotel found
713,pipili,IN,20.1167,85.8333,89,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)